In [ ]:
pip install pymc

In [ ]:
#1
import pandas as pd
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from pymc.sampling import sample

data = pd.read_csv('/content/date_alegeri_turul2.csv')

# a.
scaler = StandardScaler()
data[['varsta', 'venit']] = scaler.fit_transform(data[['varsta', 'venit']])

with pm.Model() as model_a:
    beta_varsta = pm.Normal('beta_varsta', mu=0, sigma=1)
    beta_sex = pm.Normal('beta_sex', mu=0, sigma=1)
    beta_educatie = pm.Normal('beta_educatie', mu=0, sigma=1)
    beta_venit = pm.Normal('beta_venit', mu=0, sigma=1)
    intercept = pm.Normal('intercept', mu=0, sigma=1)

    logits = (
        beta_varsta * data['varsta'] +
        beta_sex * data['sex'] +
        beta_educatie * data['educatie'] +
        beta_venit * data['venit'] +
        intercept
    )

    p = pm.Deterministic('p', pm.math.sigmoid(logits))
    y = pm.Bernoulli('y', p=p, observed=data['vot'])

    trace_a = pm.sample(1000, tune=1000, return_inferencedata=True, random_seed=42)

# b.
pm.summary(trace_a, hdi_prob=0.94)
#alegem 'Varsta' §i 'Venit' pe baza impactului semnificativ observat, valoriile medii si intervalele HDI
# c.
vars_selected = ['varsta', 'venit']
data_selected = data[vars_selected]

with pm.Model() as model_c:
    beta_varsta = pm.Normal('beta_varsta', mu=0, sigma=1)
    beta_venit = pm.Normal('beta_venit', mu=0, sigma=1)
    intercept = pm.Normal('intercept', mu=0, sigma=1)

    logits = (
        beta_varsta * data['varsta'] +
        beta_venit * data['venit'] +
        intercept
    )

    p = pm.Deterministic('p', pm.math.sigmoid(logits))
    y = pm.Bernoulli('y', p=p, observed=data['vot'])

    trace_c = pm.sample(1000, tune=1000, return_inferencedata=True, random_seed=42)

x_min, x_max = data['varsta'].min(), data['varsta'].max()
y_min, y_max = data['venit'].min(), data['venit'].max()
x, y = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

z = trace_c.posterior['intercept'].mean().item() + \
    trace_c.posterior['beta_varsta'].mean().item() * x + \
    trace_c.posterior['beta_venit'].mean().item() * y

prob = 1 / (1 + np.exp(-z))

plt.figure(figsize=(8, 6))
plt.contourf(x, y, prob, levels=np.linspace(0, 1, 11), cmap='coolwarm', alpha=0.8)
plt.colorbar(label='Probabilitatea de a vota pentru candidatul A')
plt.scatter(data['varsta'], data['venit'], c=data['vot'], cmap='coolwarm', edgecolor='k')
plt.xlabel('varsta (standardizata)')
plt.ylabel('venit (standardizat)')
plt.title('granita de decizie si HDI 94%')
plt.show()

# d.
waic_a = pm.waic(trace_a, model_a)
waic_c = pm.waic(trace_c, model_c)

loo_a = pm.loo(trace_a, model_a)
loo_c = pm.loo(trace_c, model_c)

print("\nModel complet (a):")
print(f"WAIC: {waic_a}")
print(f"LOO: {loo_a}")

print("\nModel cu 2 variabile (c):")
print(f"WAIC: {waic_c}")
print(f"LOO: {loo_c}")


In [ ]:
import pandas as pd

file_path = "/content/iris.csv"
iris_data = pd.read_csv(file_path)

iris_data.head()

In [ ]:
#2
import pandas as pd
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

file_path = "/content/iris.csv"
iris_data = pd.read_csv(file_path)

#functie pentru a aplica un model mixt gaussian (GMM) pe o caracteristica
def run_gmm_for_feature(data, feature_name, n_clusters=3):
    feature_data = (data[feature_name] - data[feature_name].mean()) / data[feature_name].std()

    with pm.Model() as gmm_model:
        weights = pm.Dirichlet("weights", a=np.ones(n_clusters))
        means = pm.Normal("means", mu=0, sigma=5, shape=n_clusters)
        sds = pm.HalfNormal("sds", sigma=2, shape=n_clusters)
        y = pm.NormalMixture("y", w=weights, mu=means, sigma=sds, observed=feature_data)
        trace = pm.sample(1000, return_inferencedata=True, progressbar=False)

        waic = pm.waic(trace, gmm_model)

    return trace, waic

features = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
results = {}

for feature in features:
    trace, waic = run_gmm_for_feature(iris_data, feature)
    results[feature] = {"trace": trace, "waic": waic}

waic_scores = {feature: results[feature]["waic"].waic for feature in features}

print("Scoruri WAIC pentru fiecare caracteristica:")
for feature, waic in waic_scores.items():
    print(f"{feature}: WAIC = {waic:.2f}")